# Practice Notebook: Writing Data to Neo4j

## Pre-requisite

In [ ]:
# importing neo4j drive
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 KB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.6.0-py3-none-any.whl size=237450 sha256=a11324af5dbd6c9eb8495f277bfca6eff79835dd7770fe6de01649a327aca117
  Stored in directory: /root/.cache/pip/wheels/c0/9b/d9/fdb6b67a6f6d7aef4acaefe55f339739caf09bb63e43bfb10e
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase

uri = "neo4j+s://f680248d.databases.neo4j.io"
username = "neo4j"
password = "CV5io6VC99r107hqaJKPcwBukDFNJ1FTh3pHit4NoCM"

driver = GraphDatabase.driver(uri, auth=(username, password))

## Examples 

### 1. Writing Node Data

When working with Neo4j, nodes are used to represent entities in a graph. Nodes are typically labeled with a descriptive label that indicates the type of entity it represents, and properties that provide additional information about that entity. Node data can be added to Neo4j using Cypher queries, and this can be done using the CREATE command.


In [ ]:
#from neo4j import GraphDatabase

#driver = GraphDatabase.driver("", auth=("", ""))

def create_person(tx, name, age):
    tx.run("CREATE (:Person {name: $name, age: $age})", name=name, age=age)

with driver.session() as session:
    session.execute_write(create_person, "Alice", 25)

The code above connects to a Neo4j database using the GraphDatabase.driver() method, passing in the Neo4j URI and authentication credentials. It then defines a create_person() function that takes a transaction object and the name and age of a person as parameters. The create_person() function runs a Cypher query to create a new node labeled as Person with name and age properties, using the tx.run() method.

The code then creates a new session using the driver.session() method, and calls the execute_write() method of the session object to execute the create_person() function, passing in the name and age values for the new person. Finally, the code closes the session and the driver using the session.close() and driver.close() methods, respectively.

#### Challenge

Suppose you are working for a telecom company and you need to create a Neo4j graph to store customer data. Write a function create_customer that takes the following parameters: name, age, phone_number, address, and plan. The function should create a node with the label "Customer" and set the provided properties using the provided parameters. 

In [ ]:
# Your code goes here
def create_customer(mx, name, age, phone_number, address, plan):
    mx.run("CREATE (:customer {name: $name, age: $age, phone_number:$phone_number, address:$address, plan:$plan})",
           name=name, age=age, phone_number=phone_number, address=address, plan=plan)

with driver.session() as session:
    session.execute_write(create_customer, "Eric", 25, "720553382", "30 Rongai" ,"Zidisha")
    session.execute_write(create_customer, "Erigo", 35, "720554482", "55 Rongai" ,"postpay")
    session.execute_write(create_customer, "Mumbo", 29, "720333382", "3009 Rongai" ,"hybrid")



#### Sample solution

In [ ]:
def create_customer(tx, name, age, phone_number, address, plan):
    tx.run("CREATE (:Customer {name: $name, age: $age, phone_number: $phone_number, address: $address, plan: $plan})", 
           name=name, age=age, phone_number=phone_number, address=address, plan=plan)

with driver.session() as session:
    session.execute_write(create_customer, "John Smith", 35, "555-555-1234", "123 Main St, Anytown USA", "Premium")

### 2. Writing Relationship Data

In a graph database, relationships are used to establish connections between nodes. Relationships are defined with a type and can contain properties. In Neo4j, relationships are directional and always have a start and an end node.

In [ ]:
def create_relationship(tx, name1, name2):
    tx.run("MATCH (a:Person {name: $name1}), (b:Person {name: $name2}) \
            CREATE (a)-[:KNOWS]->(b)", name1=name1, name2=name2)

with driver.session() as session:
    session.execute_write(create_relationship, "Alice", "Bob")

The code provided connects to a Neo4j instance and creates a relationship between two nodes. The function create_relationship takes in two parameters, name1 and name2, which are used to match nodes with the given names. It then creates a directed KNOWS relationship from name1 to name2.

The with block establishes a session with the Neo4j instance and uses the execute_write method to create a new transaction and run the create_relationship function with the given parameters. After the transaction is completed, the session and driver are closed using session.close() and driver.close() respectively.

#### Challenge

We have a telecom company that wants to create a Neo4j graph database to store information about its customers and their phone numbers. Each customer can have multiple phone numbers. Write a Python function that takes in two parameters, customer_name and phone_number, and creates a HAS_PHONE_NUMBER relationship between the customer and the phone number node.

In [ ]:
# Your code goes here
def create_relationship(tx, customer_name, phone_number):
    tx.run("MATCH (a:customer_name {customer_name: $customer_name}), (b:phone_number {phone_number: $phone_number}) \
            CREATE (a)-[:HAS_PHONE_NUMBER]->(b)", customer_name=customer_name, phone_number=phone_number)

with driver.session() as session:
    session.execute_write(create_relationship, "GMumbo", "7206677799")

#### Sample solution

In [ ]:
def create_phone_number_relationship(tx, customer_name, phone_number):
    tx.run("MATCH (c:Customer {name: $customer_name}), (p:PhoneNumber {number: $phone_number}) \
            CREATE (c)-[:HAS_PHONE_NUMBER]->(p)", customer_name=customer_name, phone_number=phone_number)

with driver.session() as session:
    session.execute_write(create_phone_number_relationship, "John Doe", "555-1234")

### 3. Writing Node and Relationship Data Together

When creating a graph in Neo4j, it is common to have both nodes and relationships. In order to create a relationship between two nodes, both nodes need to be created first. In the example code below, we create two nodes labeled "Person" and give them a name property, then create a relationship between them with the type "KNOWS". By creating nodes and relationships together in the same transaction, we ensure that the graph is created in a consistent state, and avoid potential issues with missing nodes or relationships.

In [ ]:
def create_node_relationship(tx):
    tx.run("CREATE (n1:Person {name: $name1}) "
           "CREATE (n2:Person {name: $name2}) "
           "CREATE (n1)-[:KNOWS]->(n2)",
           name1="Alice", name2="Bob")

with driver.session() as session:
    session.execute_write(create_node_relationship)

The code uses the Neo4j Python driver to connect to a Neo4j instance.. It defines a function called "create_node_relationship" that takes a transaction object as its argument. Within the function, the transaction object is used to execute a Cypher query that creates two nodes with the labels "Person" and the properties "name1" and "name2", respectively, and a relationship between the two nodes with the type "KNOWS". The names of the nodes are specified as parameters to the Cypher query.

The main part of the code then creates a session using the driver, and calls the "create_node_relationship" function execute_write function. The session is automatically closed when the with block is exited, and the driver is closed at the end of the script.

#### Challenge

Suppose we are working for a telecom company that provides phone and internet services to customers. We want to create a graph in Neo4j that represents the relationships between our customers, the phone numbers they use, and the internet plans they have subscribed to. Write a Python script that creates the following nodes and relationships:

1. Customer nodes with properties name and email
2. Phone nodes with properties number and type
3. Internet plan nodes with properties name and speed
4. Relationships between customer nodes and phone nodes with type HAS_PHONE
5. Relationships between customer nodes and internet plan nodes with type HAS_PLAN

In [ ]:
# Your code goes here
def create_customer(tx, name, email):
    tx.run("CREATE (:customer {name: $name, email: $email})",
           name=name, email=email)
def create_phone_node(tx, number, type):
    tx.run("CREATE (:number {number: $number, type: $type})",
           number=number, type=type)
def create_internet_node(tx, plan, speed):
    tx.run("CREATE (:plan {plan: $plan, speed: $speed})",
           plan=plan, speed=speed)
def create_customer_phone_rel(tx, customer_name, phone_number):
  tx.run("MATCH(c:customer{name:$customer_name}), (p:PhoneNumber {number: $phone_number}) \
            CREATE (c)-[:HAS_PHONE_NUMBER]->(p)", customer_name=customer_name, phone_number=phone_number)

def create_customer_internetplan_rel(tx, customer_name, plan):
  tx.run("MATCH(c:customer{name:$customer_name}), (k:plan {plan: $plan}) \
            CREATE (c)-[:HAS_PLAN]->(k)", customer_name=customer_name, plan=plan)






with driver.session() as session:
    session.execute_write(create_customer, "Eric", "eric@eric.com")
    session.execute_write(create_phone_node, "720777772", "home")
    session.execute_write(create_internet_node, "4GoLTE", "5mbps")
    session.execute_write(create_customer_phone_rel, "Eric", "720777772")
    session.execute_write(create_customer_internetplan_rel, "Eric", "5mbps")

#### Sample solution

In [ ]:
def create_customer_phone_plan(tx):
    tx.run("CREATE (:Customer {name: $name, email: $email})-[:HAS_PHONE]->(:Phone {number: $number, type: $type})",
           name="John Doe", email="john.doe@example.com", number="555-1234", type="mobile")
    tx.run("CREATE (:Customer {name: $name, email: $email})-[:HAS_PLAN]->(:InternetPlan {name: $plan_name, speed: $plan_speed})",
           name="Jane Smith", email="jane.smith@example.com", plan_name="Silver", plan_speed="100Mbps")

with driver.session() as session:
    session.write_transaction(create_customer_phone_plan)

<ipython-input-16-b7e610538a17>:8: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_customer_phone_plan)


In [ ]:
driver.close()